In [1]:
# Import Libraries
import pandas as pd

# Data Merging and Cleaning 

In [9]:
my_spotify_data = pd.read_csv("data/my_spotify_data.csv", encoding='latin-1', low_memory=False)
my_spotify_data.head(1)

,track_name,artist_name,album_name,spotify_track_uri,skipped,date,time,min_played
0,Heaven - Originally Performed By DJ Sammy & Yanou,It's A Cover Up,Dance Anthems,spotify:track:5V2u2roMAFN97vgTt01OlG,TRUE,9/6/2012,19:45:53,1.250816667


In [16]:
tracks = pd.read_csv("data/tracks.csv")
tracks.drop(columns=['id','id_artists'],inplace=True) # unique for each row
tracks.head(1)

,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Carve,6,126903,0,['Uli'],1922-02-22,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,3


In [17]:
from ast import literal_eval
tracks['artists'] = tracks['artists'].apply(lambda x: literal_eval(x))
tracks['artist'] = tracks['artists'].apply(lambda x: x[0])
tracks.drop(columns='artists',inplace=True)

In [19]:
data = pd.merge(my_spotify_data, tracks, left_on = ['track_name','artist_name'], right_on = ['name','artist']) # limitation: 

artist_name
Justin Bieber      12400
Drake              10052
Imagine Dragons     7259
Maroon 5            6548
Ed Sheeran          6401
                   ...  
Lane 8                 1
Nessa Barrett          1
Larissa Lambert        1
The High Kings         1
Sech                   1
Name: count, Length: 2837, dtype: int64